# Evaluating Numerical Expressions

In [79]:
from linked_list import LinkedList

In [80]:
class Stack(LinkedList):
    def push(self, data):
        self.append(data)
        
    def peek(self):
        return self.tail.data
    
    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

## Tokenize the expression

In [81]:
def tokenize(postfix_expr):
    return postfix_expr.split()

In [82]:
print(tokenize("12 2 4 + / 21 *"))

['12', '2', '4', '+', '/', '21', '*']


## Processing an Operator

In [83]:
def process_minus(stack):
    top = float(stack.pop())
    second_to_top = float(stack.pop())
    result = second_to_top - top
    stack.push(result)

In [84]:
def process_plus(stack):
    top = float(stack.pop())
    second_to_top = float(stack.pop())
    result = second_to_top + top
    stack.push(result)

In [85]:
def process_times(stack):
    top = float(stack.pop())
    second_to_top = float(stack.pop())
    result = second_to_top * top
    stack.push(result)

In [86]:
def process_divide(stack):
    top = float(stack.pop())
    second_to_top = float(stack.pop())
    result = second_to_top / top
    stack.push(result)

In [87]:
def process_pow(stack):
    top = float(stack.pop())
    second_to_top = float(stack.pop())
    result = second_to_top ** top
    stack.push(result)

In [88]:
token_actions = {
    "+": process_plus,
    "-": process_minus,
    "/": process_divide,
    "*": process_times,
    "**": process_pow
}

## Evaluating Postfix Expressions

In [89]:
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    for token in tokens:
        if token not in ("+", "-", "*", "/", "**"):
            stack.push(token)
        else:
            token_actions[token](stack)
    return stack.peek()

In [90]:
evaluate_postfix("4 6 -")

-2.0

In [91]:
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

In [92]:
for expr in expressions:
    print(evaluate_postfix(expr))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


## Operator Precedence in Infix Notation

In [93]:
precedence = {
    "+": 1,
    "-": 1, 
    "*": 2,
    "/": 2,
    "**": 3
}

In [94]:
precedence["*"] > precedence["+"]

True

## From Infix to Postfix

In [95]:
def process_opening_paranthesis(stack):
    stack.push("(")

In [96]:
def process_closing_paranthesis(stack, postfix):
    while stack.peek() != "(":
        popped = stack.pop()
        postfix.append(popped)
    stack.pop()  # pop "("

In [97]:
def process_operator(stack, postfix, operator):
    while (len(stack) > 0 and (stack.peek() in precedence) and precedence[stack.peek()] >= precedence[operator]):
        popped = stack.pop()
        postfix.append(popped)
    stack.push(operator)

In [98]:
def process_number(postfix, number):
    postfix.append(number)

### Implementing the Shunting-yard Algorithm

In [99]:
def infix_to_postfix(expression):
    stack = Stack()
    postfix = []
    tokens = expression.split()
    for token in tokens:
        if token == "(":
            process_opening_paranthesis(stack)
        elif token == ")":
            process_closing_paranthesis(stack, postfix)
        elif token in precedence:
            process_operator(stack, postfix, token)
        else:  # token is a number
            process_number(postfix, token)
    
    while len(stack) > 0:
        popped = stack.pop()
        postfix.append(popped)
    
    postfix_expr = " ".join(postfix)
    return postfix_expr

## Evaluating Infix Expressions

In [100]:
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

In [101]:
def evaluate(infix_expr):
    postfix_expr = infix_to_postfix(infix_expr)
    res = evaluate_postfix(postfix_expr)
    return res

In [102]:
for expr in expressions:
    print(evaluate(expr))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0
